In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from obspy import read
from msnoise.api import *
from wxs_dvv import *
import os
import numpy as np
import tqdm
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
plt.rcParams['figure.figsize'] = (16,8)

## Getting parameters from the msnoise DB

Like in the previous notebook, we try to take advantage of the MSNoise configuration DB to stay consistant.

In [ ]:
db = connect()
params = get_params(db)
filterid=1
fs = params.cc_sampling_rate
lag_min = params.dtt_minlag
lag_max = params.dtt_minlag+params.dtt_width
freqmin=0.1
freqmax=4.0
subdaily=False
saveit=True
plot = False
static_ref = True

stations_to_analyse = ["%s.%s.%s" % (sta.net, sta.sta, sta.locs()[0]) for sta in get_stations(db, all=False)]
pairs = ["{}:{}".format(sta,sta) for sta in stations_to_analyse]
pair = pairs[0]
start = params.startdate
end = params.enddate
comp = "EZ" # Choose your adventure
mov_stack = 30 # Choose your adventure

# Plotting the dv/v

In [ ]:
df_file = glob.glob("WCT/{}*.pkl".format(pair.replace(":","_")))[0]
dvv_df = pd.read_pickle(df_file)
dvv_df

fig, ax2 = plt.subplots(figsize=(20,5))
clim = 2
span = 30 # Smoothing alert!
ax2.pcolormesh(np.asarray(dvv_df.ewm(span = span).mean().index), 
               np.asarray(dvv_df.ewm(span = span).mean().columns), 
               dvv_df.ewm(span = span).mean().astype(float).T.values, 
               cmap='seismic_r', edgecolors='none', vmin=-clim, vmax=clim)
cmap = mpl.cm.seismic_r
norm = mpl.colors.Normalize(vmin=-clim, vmax=clim)
cbar2=plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax = ax2)
cbar2.set_label('dv/v', rotation=270)
ax2.set_ylim(freqmin, freqmax)
ax2.set_title('dv/v {} {}'.format(pair.split(":")[0],comp), fontsize=13)
ax2.set_ylabel('Frequency (Hz)')
plt.gca().xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
fig.autofmt_xdate()

#    plt.savefig("WTC/Figure/dvv_wct {} {}.png".format(pair,comp), dpi=300)
plt.show()

## Let's compare!

In [ ]:
# Load Tim's dv/v fo the station
clement = pd.read_pickle("DATA/clement.pkl")
clement.index = pd.to_datetime(clement.index)
clement.columns = ["Clements 2023"]

# We stick to the same frequency range
high = 4 
low = 2 

#Formatting the DataFrame
select = dvv_df.T
sub_select = select[(select.index>=low) & (select.index<=high)].mean().ewm(span = span).mean()

Plot it!

In [ ]:
fig, ax = plt.subplots()
sub_select.plot(ax=ax,title="dv/v variation at station {} for the component {}".format(pair.split(":")[0], comp), label="Your curve")
clement.plot(ax=ax, label="Clement 2023")
plt.legend()
plt.ylabel("dv/v (%)")
plt.show()

## Let's not stop here

Pick one or several frequency ranges to average and save.

You will use this frequency range in the next notebook to estimate the depth of the observed change

In [ ]:
high = # Choose your adventure
low = # Choose your adventure
fn = # Choose a name for your frequency range of choice


select = dvv_df.T
sub_select = select[(select.index>=low) & (test.index<=high)].mean().ewm(span = span).mean()
fig, ax = plt.subplots()
sub_select.plot(ax=ax,title="dv/v variation at station {} for the component {}".format(pair.split(":")[0], comp))
plt.ylabel("dv/v (%)")
plt.show()

sub_select.to_pickle("DATA/filtered_dvv_{}_{}_{}".format(pair.split(":")[0], comp, fn))

The error on the linear regression used at the dv/v step was also saved and should always be reviewed to assess the reliability of your measurements.

In [ ]:
df_file = glob.glob("WCT/Err*{}*.pkl".format(pair.replace(":","_")))[0]
dvv_df = pd.read_pickle(df_file)
dvv_df

fig, ax2 = plt.subplots(figsize=(20,8))
clim = 0.1
span = 40 # Smoothing alert!
cmap = mpl.cm.viridis
ax2.pcolormesh(np.asarray(dvv_df.ewm(span = span).mean().index), 
               np.asarray(dvv_df.ewm(span = span).mean().columns), 
               dvv_df.ewm(span = span).mean().astype(float).T.values, 
               cmap=cmap, edgecolors='none', vmin=0, vmax=clim)

norm = mpl.colors.Normalize(vmin=0, vmax=clim)
cbar2=plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax = ax2)
cbar2.set_label('dv/v', rotation=270)
ax2.set_ylim(0.1, 2)
ax2.set_title('Error %s'%pair, fontsize=13)
ax2.set_ylabel('Frequency (Hz)')
plt.gca().xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
fig.autofmt_xdate()

#    plt.savefig("dvv wct %s.png"%pair, dpi=300)
plt.show()